In [8]:
%use lets-plot

import org.jetbrains.letsPlot.*
import org.jetbrains.letsPlot.geom.*
import org.jetbrains.letsPlot.letsPlot
import org.jetbrains.letsPlot.ggplot

import kotlin.math.ceil
import kotlin.math.ln
import kotlin.system.measureNanoTime
import kotlin.random.Random
import kotlin.system.measureTimeMillis

In [14]:
import kotlin.math.ceil
import kotlin.math.ln

class SCGNode(var value: Int) {
    var left: SCGNode? = null
    var right: SCGNode? = null
    var parent: SCGNode? = null
    var subtreeSize: Int = 1
}

class ScapeGoatTree(private val alpha: Double = 2.0 / 3.0, private val maxSizeCoefficient: Double = 1.0 / 2.0) {
    private var root: SCGNode? = null
    private var size: Int = 0
    private var maxSize: Int = 0
    private var rebalanceCount: Int = 0

    fun getRebalanceCount(): Int = rebalanceCount

    private fun log1OverAlpha(value: Int): Int {
        return ceil(ln(value.toDouble()) / ln(1 / alpha)).toInt()
    }

    private fun getSubtreeSize(node: SCGNode?): Int = node?.subtreeSize ?: 0

    fun searchKey(key: Int): Int? {
        var current = root
        while (current != null) {
            current = when {
                key == current.value -> return current.value
                key < current.value -> current.left
                else -> current.right
            }
        }
        return null
    }

    fun insert(key: Int) {
        val newNode = SCGNode(key)
        val depth = insert(newNode)

        if (depth > log1OverAlpha(size)) {
            var scapegoat = newNode.parent
            while (scapegoat != null &&
                getSubtreeSize(scapegoat) <= alpha * getSubtreeSize(scapegoat.parent)
            ) {
                scapegoat = scapegoat.parent
            }

            if (scapegoat != null) {
                if (scapegoat.parent == null) {
                    rebuild(scapegoat)
                } else {
                    rebuild(scapegoat.parent!!)
                }
                rebalanceCount++
            }
        }
    }

    private fun insert(newNode: SCGNode): Int {
        if (root == null) {
            root = newNode
            size++
            maxSize = maxOf(maxSize, size)
            return 0
        }

        var current: SCGNode = root!!
        var depth = 0

        while (true) {
            current.subtreeSize++
            depth++

            when {
                newNode.value < current.value -> {
                    if (current.left == null) {
                        current.left = newNode
                        newNode.parent = current
                        break
                    } else {
                        current = current.left!!
                    }
                }
                newNode.value > current.value -> {
                    if (current.right == null) {
                        current.right = newNode
                        newNode.parent = current
                        break
                    } else {
                        current = current.right!!
                    }
                }
                else -> {
                    var rollback = current
                    while (rollback != newNode.parent) {
                        rollback.subtreeSize--
                        rollback = rollback.parent ?: break
                    }
                    return -1
                }
            }
        }

        size++
        maxSize = maxOf(maxSize, size)
        return depth
    }

    fun delete(key: Int) {
        if (delete(root, key) != null) {
            size--
            if (size < maxSizeCoefficient * maxSize) {
                root?.let { rebuild(it) }
                rebalanceCount++
                maxSize = size
            }
        }
    }

    private fun delete(node: SCGNode?, key: Int): SCGNode? {
        if (node == null) return null

        val updatedNode = when {
            key < node.value -> {
                node.left = delete(node.left, key)
                node.left?.parent = node
                node
            }
            key > node.value -> {
                node.right = delete(node.right, key)
                node.right?.parent = node
                node
            }
            else -> {
                when {
                    node.left == null -> node.right
                    node.right == null -> node.left
                    else -> {
                        val minValue = findMinValue(node.right!!)
                        node.value = minValue
                        node.right = delete(node.right, minValue)
                        node.right?.parent = node
                        node
                    }
                }
            }
        }

        updatedNode?.subtreeSize = 1 + getSubtreeSize(updatedNode?.left) + getSubtreeSize(updatedNode?.right)
        return updatedNode
    }

    private fun findMinValue(node: SCGNode): Int {
        var current = node
        while (current.left != null) {
            current = current.left!!
        }
        return current.value
    }

    private fun rebuild(node: SCGNode) {
        val parent = node.parent
        val nodeList = mutableListOf<SCGNode>()
        flattenTree(node, nodeList)

        val rebuilt = buildBalanced(nodeList)
        rebuilt?.parent = parent

        when {
            parent == null -> root = rebuilt
            parent.left == node -> parent.left = rebuilt
            parent.right == node -> parent.right = rebuilt
        }
    }

    private fun flattenTree(node: SCGNode?, list: MutableList<SCGNode>) {
        if (node == null) return
        flattenTree(node.left, list)
        list.add(node)
        flattenTree(node.right, list)
        node.left = null
        node.right = null
    }

    private fun buildBalanced(list: List<SCGNode>, start: Int = 0, end: Int = list.size): SCGNode? {
        if (start >= end) return null
        val mid = start + (end - start) / 2
        val node = list[mid]
        node.left = buildBalanced(list, start, mid)
        node.right = buildBalanced(list, mid + 1, end)
        node.left?.parent = node
        node.right?.parent = node
        node.subtreeSize = 1 + getSubtreeSize(node.left) + getSubtreeSize(node.right)
        return node
    }
}

In [78]:
// Параметры исследования
val maxSizeCoefficients = listOf(0.5, 0.7, 0.8, 1.0)
val treeSize = 131072

// Лучшая последовательность (сбалансированное дерево)
val bestKeys = mutableListOf<Int>()
var dif = treeSize / 2
    while (dif > 0) {
        for (v in dif..treeSize step dif) {
            bestKeys.add(v)
        }
        dif /= 2
    }
    
// Худшая последовательность (вырожденное дерево)
val worstKeys = (1..treeSize).toList()

In [90]:
val data = mutableMapOf<String, MutableList<Any>>(
    "operation" to mutableListOf(),
    "timeNs" to mutableListOf(),
    "n" to mutableListOf(),
    "maxSizeCoefficient" to mutableListOf(),
    "case" to mutableListOf(),
    "rebalanceCount" to mutableListOf()
)

// Совершаем кучу операций
for (maxSizeCoefficient in maxSizeCoefficients) {
    for ((keys, case) in listOf(bestKeys to "best", worstKeys to "worst")) {
        val tree = ScapeGoatTree(maxSizeCoefficient = maxSizeCoefficient)

        // Вставка: замеряем время после каждой 5000-й вставки
        var timeStart = System.nanoTime()
        keys.forEachIndexed { i, key ->
            tree.insert(key)

            // Каждые 1500 вставок замеряем время
            if ((i + 1) % 5000 == 0) {
                val timeEnd = System.nanoTime()
                val timeElapsed = timeEnd - timeStart
                data["operation"]!!.add("insert")
                data["timeNs"]!!.add(timeElapsed.toDouble())
                data["n"]!!.add(i + 1) 
                data["maxSizeCoefficient"]!!.add(maxSizeCoefficient)
                data["case"]!!.add(case)
                data["rebalanceCount"]!!.add(tree.getRebalanceCount())
                
                // Перезапускаем таймер
                timeStart = System.nanoTime()
            }
        }

        // Удаление
        var reversed = if (case == "best") keys.reversed() else keys.toMutableList()
        
        reversed.forEachIndexed { i, key ->
            tree.delete(key)

            if ((i + 1) % 5000 == 0) {
                val timeEnd = System.nanoTime()
                val timeElapsed = timeEnd - timeStart
                data["operation"]!!.add("delete")
                data["timeNs"]!!.add(timeElapsed.toDouble())
                data["n"]!!.add(i + 1)
                data["maxSizeCoefficient"]!!.add(maxSizeCoefficient)
                data["case"]!!.add(case)
                data["rebalanceCount"]!!.add(tree.getRebalanceCount())

                timeStart = System.nanoTime()
            }
        }

        // Поиск

        keys.forEachIndexed { i, key ->
            tree.searchKey(key)

            if ((i + 1) % 5000 == 0) {
                val timeEnd = System.nanoTime()
                val timeElapsed = timeEnd - timeStart
                data["operation"]!!.add("search")
                data["timeNs"]!!.add(timeElapsed.toDouble())
                data["n"]!!.add(i + 1)
                data["maxSizeCoefficient"]!!.add(maxSizeCoefficient)
                data["case"]!!.add(case)
                data["rebalanceCount"]!!.add(tree.getRebalanceCount())

                timeStart = System.nanoTime()
            }
        }
    }
}


In [91]:

// Цвета по коэффициентам
val coeffColors = mapOf(
    0.5 to "#1f77b4", 
    0.7 to "#ff7f0e",  
    0.8 to "#2ca02c", 
    1.0 to "#d62728"   
)

// Стили линий для лучшего и худшего случая
val caseLineTypes = mapOf(
    "best" to "solid",
    "worst" to "dashed"
)

val yLims = listOf(6000000.0, 4500000.0, 500000.0)

// Графики по операциям
listOf("insert", "delete", "search").forEachIndexed  {i, operation ->
    val indices = data["operation"]!!.withIndex()
        .filter { it.value == operation }
        .map { it.index }

    // Фильтрация данных для текущего графика
    val filteredData = mapOf(
        "n" to indices.map { (data["n"]!![it] as Int) % treeSize  },
        "timeNs" to indices.map { data["timeNs"]!![it] },
        "case" to indices.map { data["case"]!![it] },
        "maxSizeCoefficient" to indices.map { data["maxSizeCoefficient"]!![it].toString() }
    )

    // Построение графика
    val plot = letsPlot(filteredData) +
        geomLine(
            size = 0.5,
            alpha = 0.7
        ) {
            x = "n"
            y = "timeNs"
            color = "maxSizeCoefficient"
            linetype = "case"
        } +
        scaleColorManual(values = coeffColors.mapKeys { it.key.toString() }) +
        scaleLinetypeManual(values = caseLineTypes) +
        scaleYContinuous(limits = 0.0 to yLims[i]) +
        ggtitle("$operation time (ns) by number of operations") +
        ggsize(1200, 600)

    plot.show()
}

// графики в папке task2_img

**Анализ зависимостей** <br>
1) В асимптотически лучшем случае получается сильно лучше, чем в худшем
2) Insert выполняется сильно дольше остальных операций, а Search сильно быстрее.
3) Insert лучше всего выпоняется при maxSizeCoefficient=0.7 и хуже всего при maxSizeCoefficient=1
4) Delete лучше всего выпоняется при maxSizeCoefficient=0.5 и хуже всего при maxSizeCoefficient=0.8
5) Search лучше всего выпоняется при maxSizeCoefficient=1 и хуже всего при maxSizeCoefficient=0.7, 0.8